In [46]:
#training the holistic model
from tqdm import tqdm
import torch
import torch.nn as nn
from transformers import BertModel, AdamW, get_linear_schedule_with_warmup
import pandas as pd
import numpy as np
import pickle
import time
import random
import os
import sys
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [2]:
path = "/disk2/sadat/FakeNewsData/Holistic_v2/"
dirs = os.listdir(path)

In [8]:
def creating_test_set(path):
    '''This function will help to accumulate from the test datasets and build the dataset for training set
    '''
    test_dirs = [f for f in os.listdir(path) if f.find("test")==0] #All the testing data
    dataframes = []
    for t in test_dirs: #delete the slicing part
        df = pd.read_pickle(path+t)
        dataframes.append(df[["tokenized", "is_deception"]])
    concat_dataframe = pd.concat(dataframes, axis=0)
    concat_dataframe.reset_index(drop=True, inplace=True)
    test_text = np.array(list(concat_dataframe["tokenized"]))
    attention_masks = np.where(test_text>0, 1, 0)
    test_labels = np.array(list(concat_dataframe["is_deception"]))
    # Convert other data types to torch.Tensor
    test_text, attention_masks, test_labels = torch.tensor(test_text), torch.tensor(attention_masks), torch.tensor(test_labels)
    return test_text, attention_masks, test_labels

In [9]:
test_text, attention_masks, test_labels = creating_test_set(path)

In [22]:
model_path = path + "bert_holistic_epoch_2.pth"

In [25]:
model = torch.load(model_path)

In [31]:
#Create the DataLoader for our test set
test_dataset = TensorDataset(test_text, attention_masks)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=16)

In [53]:
import torch.nn.functional as F

def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    all_logits = []

    # For each batch in our test set...
    for batch in tqdm(test_dataloader):
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    probs = F.softmax(all_logits, dim=1).cpu().numpy()

    return probs

In [51]:
device = torch.device("cpu")

In [45]:
%%time

# Create the BertClassfier class
class BertClassifier(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, freeze_bert=False):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        """
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 50, 2

        # Instantiate BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            #nn.Dropout(0.5),
            nn.Linear(H, D_out)
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

CPU times: user 68 µs, sys: 0 ns, total: 68 µs
Wall time: 73.2 µs


In [47]:
model = BertClassifier()
model.load_state_dict(torch.load(model_path))
model.eval()

BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [54]:
probs = bert_predict(model, test_dataloader)

100%|██████████| 754/754 [1:04:20<00:00,  5.12s/it]


In [58]:
import sklearn
def creating_test_dataframe(path, probs):
    '''This function will help to accumulate from the test datasets, add the predicted labels.
    '''
    test_dirs = [f for f in os.listdir(path) if f.find("test")==0] #All the testing data
    dataframes = []
    for t in test_dirs: #delete the slicing part
        df = pd.read_pickle(path+t)
        dataframes.append(df)
    concat_dataframe = pd.concat(dataframes, axis=0)
    concat_dataframe.reset_index(drop=True, inplace=True)
    concat_dataframe["probs_0"] = probs[:,0]
    concat_dataframe["probs_1"] = probs[:,1]
    concat_dataframe["prediction"] = concat_dataframe.probs_0.apply(lambda x:0 if x>0.5 else 1)
    return concat_dataframe



In [63]:
from sklearn.metrics import f1_score, accuracy_score

In [61]:
path = "/disk2/sadat/FakeNewsData/Holistic_v2/"
test_set = creating_test_dataframe(path, probs)
f1_score(test_set["is_deception"].values, test_set["prediction"].values)
test_set.to_pickle(path + "holistic_prediction_test_set.pkl")

In [62]:
f1_score(test_set["is_deception"].values, test_set["prediction"].values)

0.8901413189771197

In [64]:
grps = test_set.groupby(["group", "subgroup"])
for name, group in grps:
    accuracy = sklearn.metrics.accuracy_score(group["is_deception"].values, group["prediction"].values)
    F1_score = f1_score(group["is_deception"].values, group["prediction"].values)
    print("group name:  ", name)
    print("\nsize of this group is:  ", group.shape[0])
    
    print("\n\n")
    print("accuracy is {}, and F1 score is {}\n".format(accuracy*100, F1_score*100))
    print("--------------------------")

group name:   ('COVID', 'AAAI')

size of this group is:   2140



accuracy is 96.16822429906541, and F1 score is 95.88766298896691

--------------------------
group name:   ('COVID', 'zenodo')

size of this group is:   1020



accuracy is 96.96078431372548, and F1 score is 98.40780688238316

--------------------------
group name:   ('EMNLP2017', 'Newsfiles')

size of this group is:   3886



accuracy is 99.63973237261966, and F1 score is 99.71544715447155

--------------------------
group name:   ('EMNLP2017', 'Politifact')

size of this group is:   712



accuracy is 81.60112359550563, and F1 score is 71.58351409978307

--------------------------
group name:   ('FakeNewsNet', 'Politifact')

size of this group is:   106



accuracy is 83.01886792452831, and F1 score is 79.54545454545455

--------------------------
group name:   ('FakeNewsNet', 'gossipcop')

size of this group is:   2214



accuracy is 85.86269196025293, and F1 score is 66.94825765575501

--------------------------
grou